# Contrução do Calendário
Como os registros de vendas que serão utilizados neste estudos são de empresas norte americanas, iremos agregar as datas de feriados e datas especiais ao dataframe de vendas e entender o comportamento das vendas nestes períodos.<br>

O dados serão adquiridos utilizando a técnica de webscraping no site timeanddate.com, buscando pelo intervalo de 2014 à 2018 (intervalo de registros contidos no dataframe)


#### Imports

In [63]:
import locale
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime


#### Utilizando beatifulSoup para extrair tabela de calendário

In [47]:
data = []
# Como o time_range do dataframe é de 2014 a 2018, vamos buscar por esses calendários
for year in range(2014,2019):
    # TODO Buscar informações sobre o pacote request e sobre a função get
    #? explicar sobre a manipulação dos anos na string da url
    page = (
        requests
            .get(f'https://www.timeanddate.com/holidays/us/{year}?hol=25')
    )

    #TODO Buscar informações sobre o BS e sobre as seguintes funções: html.parser, find, find_all
    #?  explicar sobre as tags htmls utilizadas na pesquisa.
    soup = bs(page.text,'html.parser')

    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all(['td','th'])
        #? explicar o porque da comparação do comprimento
        cols = (
            [
                f'{el.text.strip()} de {year}' 
                if len(el.text.strip().split('de')) > 1 
                else el.text.strip() 
                for el in cols
                ]
        )
        data.append([el for el in cols if el])
    
calendario_us = (
    pd.DataFrame(
        data
        , columns = [
            'DATA'
            ,'DIA_DA_SEMANA'
            ,'FERIADO'
            , 'TIPO_FERIADO'
            , 'DETALHES'
            ]
    )
)

#### Organizando o dataset do calendário


In [48]:
# Mostrando as 5 primeiras linhas do dataframe
calendario_us.head()


,DATA,DIA_DA_SEMANA,FERIADO,TIPO_FERIADO,DETALHES
0,None,None,None,None,None
1,1 de Jan de 2014,Quarta-feira,New Year's Day,Federal Holiday de 2014,None
2,4 de Jan de 2014,Sábado,World Braille Day,Worldwide observance de 2014,None
3,6 de Jan de 2014,Segunda-feira,Epiphany,Christian,None
4,7 de Jan de 2014,Terça-feira,Orthodox Christmas Day,Orthodox,None


Vamos Inicialmente checar os valores missing neste dataframe


In [49]:
# Observando a quantidade de valores None por coluna
calendario_us.isna().sum()


DATA               60
DIA_DA_SEMANA      60
FERIADO            60
TIPO_FERIADO       60
DETALHES         1812
dtype: int64

In [50]:
# Como a data é a coluna mais importante deste dataframe, vamos dropar todas as linhas onde a data seja None.
(
    calendario_us
        .dropna(
            subset = ['DATA']
            , inplace = True
        )
)


In [51]:
# Validando
calendario_us.isna().sum()


DATA                0
DIA_DA_SEMANA       0
FERIADO             0
TIPO_FERIADO        0
DETALHES         1752
dtype: int64

Após removermos todas as linhas com dados datas faltantes, mudaremos formato de apresentação da coluna calendario_us['DATA']

In [52]:
# Como o formato da data esta especificamente no formato pt_BR, foi necessário mudar o padrão POSIX do python para facilitar a atividade de conversão.
locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

'pt_BR.UTF-8'

In [53]:
# Modificando o formato das datas
calendario_us['DATA'] = (
    pd.to_datetime(
        calendario_us['DATA']
            .apply(
                lambda x: 
                datetime.strptime(x, '%d de %b de %Y')
            )
        ,format= "%d-%m-%Y"
    )
)


Existem certos feriados que são específicos de um estado ou de uma região dos Estados Unidos, esta informação pode ser obtida na coluna de DETALHES no dataframe calendario_us. Todas as vezes em que a coluna DETALHES for diferente de `None` o feriado é regional ou local.<br>
Como não temos informações sobre a região de onde foram feitas as ordens de compra, iremos remover todos os feriados onde a coluna de DETALHES seja diferente de None.

In [81]:
# Mostrando os feriados locais e regionais
calendario_us[calendario_us['DETALHES'].notna()]

,DATA,DIA_DA_SEMANA,FERIADO,TIPO_FERIADO,DETALHES
6,2014-01-08,Quarta-feira,Battle of New Orleans,State Legal Holiday,Louisiana
11,2014-01-17,Sexta-feira,Lee-Jackson Day,State Holiday,Virginia
13,2014-01-19,Domingo,Robert E. Lee's Birthday,State Legal Holiday,Florida
14,2014-01-19,Domingo,Confederate Heroes' Day de 2014,State Holiday,Texas
16,2014-01-20,Segunda-feira,Robert E. Lee's Birthday,State Holiday,"AL, AR, MS"
...,...,...,...,...,...
2487,2018-12-24,Segunda-feira,Christmas Eve,State Holiday,Arkansas
2489,2018-12-24,Segunda-feira,Washington's Birthday,State Holiday,"Georgia, Indiana"
2490,2018-12-25,Terça-feira,Christmas Day,State Holiday,All
2493,2018-12-26,Quarta-feira,Day After Christmas Day,State Holiday,"KS, NC, SC, TX"


In [82]:
# Criando um dataframe apenas com os feriados gerais
calendario_us = calendario_us[calendario_us['DETALHES'].isna()]

### Exportando o dataframe do calendário devidamente organizado para ser utilizado no próximo notebook

In [84]:
calendario_us.to_csv('./dataset/calendario_us.csv',header=True, sep=";", index=False, encoding="utf-8")